# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [3]:
spark = SparkSession.builder.appName('Data Frames Quiz').getOrCreate()

df = spark.read.json('../data/sparkify_log_small.json')

# Question 1

Which page did user id "" (empty string) NOT visit?

In [19]:
# TODO: write your code to answer question 1
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [21]:
empty_pages = df.filter(df.userId == '').select(col('page')).alias('empty_page').dropDuplicates()
all_pages = df.select('page').dropDuplicates()
set(all_pages.collect()) - set(empty_pages.collect())

{Row(page='Downgrade'),
 Row(page='Error'),
 Row(page='Logout'),
 Row(page='NextSong'),
 Row(page='Save Settings'),
 Row(page='Settings'),
 Row(page='Submit Downgrade'),
 Row(page='Submit Upgrade'),
 Row(page='Upgrade')}

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [20]:
# TODO: use this space to explore the behavior of the user with an empty string

# Question 3

How many female users do we have in the data set?

In [22]:
# TODO: write your code to answer question 3
df.filter(df.gender == 'F').select('userId', 'gender').dropDuplicates().count()

462

In [34]:
# Number of male and female users
df.select('userId', 'gender').dropDuplicates().groupBy('gender').agg({'gender': 'count'}).dropna().show()

+------+-------------+
|gender|count(gender)|
+------+-------------+
|     F|          462|
|     M|          501|
+------+-------------+



# Question 4

How many songs were played from the most played artist?

In [36]:
# TODO: write your code to answer question 4
df.filter(df.page == 'NextSong')\
    .select('Artist')\
    .groupBy('Artist')\
    .agg({'Artist': 'count'})\
    .withColumnRenamed('count(Artist)', 'ArtistCount')\
    .sort(desc('ArtistCount'))\
    .show(1)

+--------+-----------+
|  Artist|ArtistCount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [37]:
# TODO: write your code to answer question 5

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

